1. pip install gym
2. pip install torch torchvision
3. pip install box2d-py
4. pip install atari-py

In [16]:
import numpy as np
import gym
import numpy as np

env = gym.make('LunarLander-v2')

/home/aditya/.conda/envs/py3k/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [35]:
import torch
import torch.nn as nn
class DQN(nn.Module):
    def __init__(self, in_features, n_actions):
        super(DQN,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,n_actions)
        )
        
    def forward(self,x):
        return self.model(x.float())

In [36]:
import collections
class ExperienceBuffer():
    def __init__(self,capacity):
        self.exp_buffer = collections.deque(maxlen=capacity)
        
    def append(self,exp):
        self.exp_buffer.append(exp)
    
    def __len__(self):
        return len(self.exp_buffer)
    
    def clear(self):
        self.exp_buffer.clear()
        
    def sample(self,batch_size):
        indices = np.random.choice( range(len(self.exp_buffer)), batch_size )
        states,actions,rewards,dones,next_states = zip(*[self.exp_buffer[i] for i in indices])
        return np.array(states),np.array(actions),np.array(rewards),np.array(dones),np.array(next_states)

In [64]:
class Agent():
    def __init__(self,env,buffer):
        self.env = env
        self.buffer = buffer
        self._reset()
        
    def _reset(self):
        self.state = env.reset()
        self.total_rewards = 0.0
    
    def step(self, net, eps, device="cpu"):
        done_reward= None
        if np.random.random() < eps:
            action = env.action_space.sample()
        else:
            state_prev = torch.tensor(self.state)
            action = int(torch.argmax(net(state_prev).to(device)))
            
        state_prev = self.state
        self.state,reward,done,info = env.step(action)
        self.total_rewards+=reward
        
        self.buffer.append((state_prev,action,reward,done,self.state))
        if done:
            done_reward = self.total_rewards
            self._reset()
        return done_reward